In [8]:
import sys, os
sys.path.append(os.path.abspath(".."))


In [9]:
import pandas as pd
from pathlib import Path

from src.ofi_features import add_ofi_features, add_return_and_labels

df = pd.read_csv("../data/processed/lob_trades_resampled.csv")
df.head()


,timestamp,bid_price,bid_size,ask_price,ask_size,buy_volume,sell_volume,mid_price
0,2025-01-01 09:30:30,99.993047,27,100.013047,48,45.0,0.0,100.003047
1,2025-01-01 09:30:31,99.982647,68,100.002647,53,0.0,29.0,99.992647
2,2025-01-01 09:30:32,99.990152,46,100.010152,31,0.0,9.0,100.000152
3,2025-01-01 09:30:33,99.999557,40,100.019557,80,0.0,39.0,100.009557
4,2025-01-01 09:30:34,99.980047,72,100.000047,62,8.0,0.0,99.990047


In [10]:
df_with_ofi = add_ofi_features(df, window_short=5, window_long=20)

df_with_ofi[["buy_volume", "sell_volume", "ofi", "ofi_sum_5", "ofi_sum_20", "ofi_norm"]].head()


,buy_volume,sell_volume,ofi,ofi_sum_5,ofi_sum_20,ofi_norm
0,45.0,0.0,45.0,45.0,45.0,1.0
1,0.0,29.0,-29.0,16.0,16.0,-1.0
2,0.0,9.0,-9.0,7.0,7.0,-1.0
3,0.0,39.0,-39.0,-32.0,-32.0,-1.0
4,8.0,0.0,8.0,-24.0,-24.0,1.0


In [11]:
df_labeled = add_return_and_labels(df_with_ofi, horizon=10, threshold=0.0001)

df_labeled[["mid_price", "mid_price_future", "ret_future", "direction"]].head()


,mid_price,mid_price_future,ret_future,direction
0,100.003047,99.975237,-0.000278,-1
1,99.992647,99.983015,-0.000096,0
2,100.000152,99.983675,-0.000165,-1
3,100.009557,99.994947,-0.000146,-1
4,99.990047,99.999623,0.000096,0


In [12]:
df_model = df_labeled.dropna(subset=["ret_future"]).reset_index(drop=True)
df_model.head()


,timestamp,bid_price,bid_size,ask_price,ask_size,buy_volume,sell_volume,mid_price,ofi,ofi_sum_5,ofi_sum_20,total_volume,ofi_norm,mid_price_future,ret_future,direction
0,2025-01-01 09:30:30,99.993047,27,100.013047,48,45.0,0.0,100.003047,45.0,45.0,45.0,45.0,1.0,99.975237,-0.000278,-1
1,2025-01-01 09:30:31,99.982647,68,100.002647,53,0.0,29.0,99.992647,-29.0,16.0,16.0,29.0,-1.0,99.983015,-0.000096,0
2,2025-01-01 09:30:32,99.990152,46,100.010152,31,0.0,9.0,100.000152,-9.0,7.0,7.0,9.0,-1.0,99.983675,-0.000165,-1
3,2025-01-01 09:30:33,99.999557,40,100.019557,80,0.0,39.0,100.009557,-39.0,-32.0,-32.0,39.0,-1.0,99.994947,-0.000146,-1
4,2025-01-01 09:30:34,99.980047,72,100.000047,62,8.0,0.0,99.990047,8.0,-24.0,-24.0,8.0,1.0,99.999623,0.000096,0


In [13]:
Path("../data/processed").mkdir(parents=True, exist_ok=True)
df_model.to_csv("../data/processed/ofi_features_labels.csv", index=False)
